In [13]:
import os
import pandas as pd

# Directory paths and variables
dir = "C:/Users/asobchen/AppData/Local/Programs/Python/Python39/alberta/"
data_directory = "C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/"
stations_path = os.path.join(dir, 'Nearby_updated_2.csv')

# Directory paths and variables
stations_path_rep = os.path.join(dir, 'Nearby_updated_111.csv')
output_file = "C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/filled//re.csv"
# Ensure the output directory exists
output_dir = "C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/filled/"
os.makedirs(output_dir, exist_ok=True)


# Read the Nearby stations CSV file using stations_path
wind_turbines = pd.read_csv(stations_path)
wind_turbines_rep = pd.read_csv(stations_path_rep)


# turbine_name = 'Halkirk_Wind_Power_Facility'
turbine_name = 'Enmax_Taber'

def get_project_name(wind_turbines):
    try:
        project_name = wind_turbines.loc[
            wind_turbines['Asset Name'].str.contains(turbine_name, case=False),
            'Nearby_Station'
        ].values[0]

    except IndexError:
        raise ValueError(f"Turbine name '{turbine_name}' not found in the dataset")
    return project_name

project_name = get_project_name(wind_turbines)
print('Station name - ', project_name)

rep_stations = []
def get_project_name_near(data):
    for index, row in data.iterrows():
        if row['Asset Name'] == turbine_name and row['Nearby_Station'] != project_name:
            # App return first found station
            rep_stations.append( row['Nearby_Station'])

get_project_name_near(wind_turbines_rep)

print('Station replicant name - ', rep_stations)

wind_speed_col = 'Wind Spd (km/h)'
temp_col = 'Temp (°C)'

years = [str(year) for year in range(2020, 2024)]

def get_gap_files(name):
    [files] = [
        os.path.join(data_directory, f) for f in os.listdir(data_directory)
        if name in f and year in f and f.endswith('.csv')]
    return files

data_files = []
replicant_files = []
replicant_files_sec = []


for year in years:
    # Collect all relevant data files for the turbine project for the specific year

    data_files.append(get_gap_files(project_name))
    replicant_files.append(get_gap_files(rep_stations[0]))
    replicant_files_sec.append(get_gap_files(rep_stations[1]))


print('Files with data ', data_files)
print('Files with data for replica', replicant_files)
print('Files with data for replica second', replicant_files_sec)

# Initialize last known value and time for interpolation
last_known_wt = None
last_known_time = None  # Track the time associated with the last known value

def get_replica(row, found_column):
    global last_known_wt, last_known_time  # Access the global variables to store the last known valid value and time
    wt = ''

    # Try to get the value from the first dataset
    wt_values = rep_data_first.loc[rep_data_first["Date/Time (LST)"] == row["Date/Time (LST)"], found_column].values
    
    # Check if a valid value was found in rep_data_first
    if wt_values.size > 0 and not pd.isna(wt_values[0]):
        wt = wt_values[0]
        print(f'{found_column} from first source -', wt)
    else:
        # If not found or NaN in the first dataset, check the second dataset
        wt_values = rep_data_sec.loc[rep_data_sec["Date/Time (LST)"] == row["Date/Time (LST)"], found_column].values
        if wt_values.size > 0 and not pd.isna(wt_values[0]):
            wt = wt_values[0]
            print(f'{found_column} from second source -', wt)
        else:
            wt = ''
    
    return wt

def set_interpolation(data, column):
    # Convert column to numeric, setting errors='coerce' to handle non-numeric values gracefully
    data[column] = pd.to_numeric(data[column], errors='coerce')
    # Now interpolate
    data[column] = data[column].interpolate(method='linear', limit_direction='both')
    return data

for fi in range(len(data_files)):
    print(data_files[fi])
    print(replicant_files[fi])
    print(replicant_files_sec[fi])

    true_data = pd.read_csv(data_files[fi])
    rep_data_first = pd.read_csv(replicant_files[fi])
    rep_data_sec = pd.read_csv(replicant_files_sec[fi])

    # Loop through each row using iterrows and fill missing values
    for index, row in true_data.iterrows():
        if pd.isna(row[wind_speed_col]):
            wind_speed_value = get_replica(row, "Wind Spd (km/h)")
            true_data.at[index, wind_speed_col] = wind_speed_value
            # print(f"Filled {wind_speed_col} at index {index}: {wind_speed_value}")

    for index, row in true_data.iterrows():
        if pd.isna(row[temp_col]):
            temp_value = get_replica(row, "Temp (°C)")
            true_data.at[index, temp_col] = temp_value
            print(f"Filled {temp_col} at index {index}: {temp_value}")

    print('interpolation Wind Spd (km/h) start ---')
    
    true_interpol_w = set_interpolation(true_data, "Wind Spd (km/h)")

    print('interpolation Temp (°C) start ---')

    true_interpol = set_interpolation(true_interpol_w, "Temp (°C)")


    # Check for remaining NaNs after filling
    print("Remaining NaNs in wind speed:", true_interpol[wind_speed_col].isna().sum())
    print("Remaining NaNs in temperature:", true_interpol[temp_col].isna().sum())

    print('index ---', fi)

    # Save the DataFrame to the result directory
    file_path = data_files[fi]  # Access the file path at the current index
    base_filename = os.path.splitext(os.path.basename(file_path))[0]  # Extract base filename without extension
    output_path = os.path.join(output_dir, f"{base_filename}_filled.csv")
    true_interpol.to_csv(output_path, index=False)
    print(f"File saved to: {output_path}")

Station name -  WRENTHAM AGDM
Station replicant name -  ['FINCASTLE AGDM', 'FOREMOST AGDM']
Files with data  ['C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/WRENTHAM AGDM_2020.csv', 'C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/WRENTHAM AGDM_2021.csv', 'C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/WRENTHAM AGDM_2022.csv', 'C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/WRENTHAM AGDM_2023.csv']
Files with data for replica ['C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/FINCASTLE AGDM_2020.csv', 'C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/FINCASTLE AGDM_2021.csv', 'C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/FINCASTLE AGDM_2022.csv', 'C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/FINCASTLE AGDM_2023.csv']
Files with data for replica second ['C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/FOREMOST AGDM_2020.csv', 'C:/Users/a

C:\Users\asobchen\AppData\Local\Temp\ipykernel_2104\1653140370.py:130: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  true_data.at[index, temp_col] = temp_value


Filled Temp (°C) at index 3885: 
interpolation Wind Spd (km/h) start ---
interpolation Temp (°C) start ---
Remaining NaNs in wind speed: 0
Remaining NaNs in temperature: 0
index --- 0
File saved to: C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/filled/WRENTHAM AGDM_2020_filled.csv
C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/WRENTHAM AGDM_2021.csv
C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/FINCASTLE AGDM_2021.csv
C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/FOREMOST AGDM_2021.csv
Wind Spd (km/h) from first source - 10
Temp (°C) from first source - 21.2
Filled Temp (°C) at index 5322: 21.2
interpolation Wind Spd (km/h) start ---
interpolation Temp (°C) start ---
Remaining NaNs in wind speed: 0
Remaining NaNs in temperature: 0
index --- 1
File saved to: C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/filled/WRENTHAM AGDM_2021_filled.csv
C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nea

C:\Users\asobchen\AppData\Local\Temp\ipykernel_2104\1653140370.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  true_data.at[index, wind_speed_col] = wind_speed_value


Wind Spd (km/h) from second source - 17.0
Wind Spd (km/h) from second source - 12.0
Wind Spd (km/h) from first source - 16.0
Wind Spd (km/h) from first source - 15.0
Wind Spd (km/h) from first source - 10.0
Wind Spd (km/h) from first source - 15.0
Wind Spd (km/h) from first source - 9.0
Wind Spd (km/h) from first source - 8.0
Wind Spd (km/h) from first source - 12.0
Wind Spd (km/h) from first source - 7.0
Wind Spd (km/h) from first source - 8.0
Wind Spd (km/h) from first source - 17.0
Wind Spd (km/h) from first source - 15.0
Wind Spd (km/h) from first source - 12.0
Wind Spd (km/h) from first source - 7.0
Wind Spd (km/h) from first source - 7.0
Wind Spd (km/h) from first source - 5.0
Wind Spd (km/h) from first source - 13.0
Wind Spd (km/h) from first source - 7.0
Wind Spd (km/h) from first source - 10.0
Wind Spd (km/h) from first source - 9.0
Wind Spd (km/h) from first source - 7.0
Wind Spd (km/h) from first source - 9.0
Wind Spd (km/h) from first source - 11.0
Wind Spd (km/h) from firs

...

Filled Temp (°C) at index 2532: 
Filled Temp (°C) at index 2533: 
Filled Temp (°C) at index 2534: 
Filled Temp (°C) at index 2535: 
Filled Temp (°C) at index 2536: 
Filled Temp (°C) at index 2537: 
Filled Temp (°C) at index 2538: 
Filled Temp (°C) at index 2539: 
Filled Temp (°C) at index 2540: 
Filled Temp (°C) at index 2541: 
Filled Temp (°C) at index 2542: 
Filled Temp (°C) at index 2543: 
Filled Temp (°C) at index 2544: 
Filled Temp (°C) at index 2545: 
Filled Temp (°C) at index 2546: 
Filled Temp (°C) at index 2547: 
Filled Temp (°C) at index 2548: 
Filled Temp (°C) at index 2549: 
Filled Temp (°C) at index 2550: 
Filled Temp (°C) at index 2551: 
Filled Temp (°C) at index 2552: 
Filled Temp (°C) at index 2553: 
Filled Temp (°C) at index 2554: 
Filled Temp (°C) at index 2555: 
Filled Temp (°C) at index 2556: 
Filled Temp (°C) at index 2557: 
Filled Temp (°C) at index 2558: 
Filled Temp (°C) at index 2559: 
Filled Temp (°C) at index 2560: 
Filled Temp (°C) at index 2561: 
Filled Tem

C:\Users\asobchen\AppData\Local\Temp\ipykernel_2104\1653140370.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  true_data.at[index, wind_speed_col] = wind_speed_value


Wind Spd (km/h) from first source - 15.0
Wind Spd (km/h) from first source - 22.0
Wind Spd (km/h) from first source - 30.0
Wind Spd (km/h) from first source - 33.0
Wind Spd (km/h) from first source - 36.0
Wind Spd (km/h) from first source - 38.0
Wind Spd (km/h) from first source - 40.0
Wind Spd (km/h) from first source - 38.0
Wind Spd (km/h) from first source - 40.0
Wind Spd (km/h) from first source - 35.0
Wind Spd (km/h) from first source - 40.0
Wind Spd (km/h) from first source - 38.0
Wind Spd (km/h) from first source - 37.0
Wind Spd (km/h) from first source - 29.0
Wind Spd (km/h) from first source - 45.0
Wind Spd (km/h) from first source - 46.0
Wind Spd (km/h) from first source - 30.0
Wind Spd (km/h) from first source - 25.0
Wind Spd (km/h) from first source - 34.0
Wind Spd (km/h) from first source - 30.0
Wind Spd (km/h) from first source - 33.0
Wind Spd (km/h) from first source - 33.0
Wind Spd (km/h) from first source - 25.0
Wind Spd (km/h) from first source - 27.0
Wind Spd (km/h) 

C:\Users\asobchen\AppData\Local\Temp\ipykernel_2104\1653140370.py:130: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  true_data.at[index, temp_col] = temp_value


Filled Temp (°C) at index 3716: 
Filled Temp (°C) at index 3717: 
Filled Temp (°C) at index 3718: 
Filled Temp (°C) at index 3719: 
Filled Temp (°C) at index 3720: 
Filled Temp (°C) at index 3721: 
Filled Temp (°C) at index 3722: 
Filled Temp (°C) at index 3723: 
Filled Temp (°C) at index 3724: 
Filled Temp (°C) at index 3725: 
Filled Temp (°C) at index 3726: 
Filled Temp (°C) at index 3727: 
Filled Temp (°C) at index 3728: 
Filled Temp (°C) at index 3932: 
Filled Temp (°C) at index 4361: 
Filled Temp (°C) at index 4362: 
Filled Temp (°C) at index 4363: 
Filled Temp (°C) at index 4364: 
Filled Temp (°C) at index 4365: 
Filled Temp (°C) at index 4366: 
Filled Temp (°C) at index 4367: 
Filled Temp (°C) at index 4368: 
Filled Temp (°C) at index 4369: 
Filled Temp (°C) at index 4370: 
Filled Temp (°C) at index 4371: 
Filled Temp (°C) at index 4372: 
Filled Temp (°C) at index 4373: 
Filled Temp (°C) at index 4374: 
Filled Temp (°C) at index 4375: 
Filled Temp (°C) at index 4376: 
Filled Tem